In [8]:
!pip install datasets
!pip install transformers==4.30
!pip install transformers[torch]
!pip install accelerate -U
!pip install nltk


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Dataset Description



Data: https://data.world/opensnippets/bbc-uk-news-dataset

The dataset that I will be using for this homework will be a BBC UK news dataset with roughly 16,000 entries. The dataset provides tags, titles, publication dates, authors, language, and of course the actual content of the articles and their summaries. The Dataset is however somewhat old, being createdd three years ago.

# Data Analysis and Processing


In [9]:
import os

import json
import pandas as pd
import numpy as np
import torch

import datasets
from datasets import Dataset, DatasetDict

from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from sklearn.model_selection import train_test_split

In [10]:
file_path = 'c:\\Users\\James Pringle\\Desktop\\Projects in Machine Learning and AI\\Homeworks\\hw6\\bbc_news_list_uk.json'

with open(file_path, 'r') as file:
    data = json.load(file)

df = pd.DataFrame(data) #convert the json file in to a pandas dataframe

In [11]:
df.keys()

Index(['tags', 'title', 'news_post_date', 'raw_content', 'content', 'url',
       'author', 'language', '_id', 'region', 'short_description', 'category',
       'crawled_at'],
      dtype='object')

In [12]:
df = df[['content', 'short_description']] # Isolate the relevant columns
df = df.dropna() # drop rows with empty values
df = df.sample(n=100) #sample 10 rows from the entire dataframe to limit training time

In [13]:
# reset index and drop old index column
df.reset_index(drop=True, inplace=True)
train_df, test_df = train_test_split(df, test_size=0.1)


# convert the dataframe into a dataset for hugging face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Model



In [14]:
# define a tokenizer to tokenize the datasets
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

# Tokenize the input data
def tokenize(example):
    content = tokenizer(example['content'], truncation=True, padding='max_length', max_length=512)
    summary = tokenizer(example['short_description'], truncation=True, padding='max_length', max_length=128)

    content = {
        'input_ids': content['input_ids'],
        'attention_mask': content['attention_mask'],
        'labels': summary['input_ids']
    }

    return content

# Tokenize train dataset
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

Map: 100%|██████████| 10/10 [00:00<00:00, 192.31 examples/s]


In [15]:
# initialize the model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

# set the hyperparameters
training_args = Seq2SeqTrainingArguments(
    per_device_train_batch_size=7,
    per_device_eval_batch_size=7,
    output_dir='./results',
    logging_dir='./logs',
    num_train_epochs=2,
    logging_steps=1000,
    evaluation_strategy='steps',
    eval_steps=1000,
    warmup_steps=200,
    weight_decay=0.05,
    predict_with_generate=True
)

# initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

c:\Users\James Pringle\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 26/26 [1:01:30<00:00, 141.96s/it]

{'train_runtime': 3690.8359, 'train_samples_per_second': 0.049, 'train_steps_per_second': 0.007, 'train_loss': 13.13360830453726, 'epoch': 2.0}


TrainOutput(global_step=26, training_loss=13.13360830453726, metrics={'train_runtime': 3690.8359, 'train_samples_per_second': 0.049, 'train_steps_per_second': 0.007, 'train_loss': 13.13360830453726, 'epoch': 2.0})

In [16]:
evaluation_results = trainer.evaluate()
print(evaluation_results)

100%|██████████| 2/2 [00:16<00:00,  8.15s/it]

{'eval_loss': 11.37539005279541, 'eval_runtime': 54.5248, 'eval_samples_per_second': 0.183, 'eval_steps_per_second': 0.037, 'epoch': 2.0}


## BLEU : Bilingual Evaluation Understudy Score

In [17]:
import nltk
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu


def bleu(ref, gen):
    ''' 
    calculate pair wise bleu score. uses nltk implementation
    Args:
        references : a list of reference sentences 
        candidates : a list of candidate(generated) sentences
    Returns:
        bleu score(float)
    '''
    ref_bleu = []
    gen_bleu = []
    for l in gen:
        gen_bleu.append(l.split())
    for i,l in enumerate(ref):
        ref_bleu.append([l.split()])
    cc = SmoothingFunction()
    score_bleu = corpus_bleu(ref_bleu, gen_bleu, weights=(0, 1, 0, 0), smoothing_function=cc.method4)
    return score_bleu


## Rouge: Recall Oriented Understudy for Gisting Evaluation

In [18]:
#rouge scores for a reference/generated sentence pair
#source google seq2seq source code.

import itertools

#supporting function
def _split_into_words(sentences):
  """Splits multiple sentences into words and flattens the result"""
  return list(itertools.chain(*[_.split(" ") for _ in sentences]))

#supporting function
def _get_word_ngrams(n, sentences):
  """Calculates word n-grams for multiple sentences.
  """
  assert len(sentences) > 0
  assert n > 0

  words = _split_into_words(sentences)
  return _get_ngrams(n, words)

#supporting function
def _get_ngrams(n, text):
  """Calcualtes n-grams.
  Args:
    n: which n-grams to calculate
    text: An array of tokens
  Returns:
    A set of n-grams
  """
  ngram_set = set()
  text_length = len(text)
  max_index_ngram_start = text_length - n
  for i in range(max_index_ngram_start + 1):
    ngram_set.add(tuple(text[i:i + n]))
  return ngram_set

def rouge_n(reference_sentences, evaluated_sentences, n=2):
  """
  Computes ROUGE-N of two text collections of sentences.
  Source: http://research.microsoft.com/en-us/um/people/cyl/download/
  papers/rouge-working-note-v1.3.1.pdf
  Args:
    evaluated_sentences: The sentences that have been picked by the summarizer
    reference_sentences: The sentences from the referene set
    n: Size of ngram.  Defaults to 2.
  Returns:
    recall rouge score(float)
  Raises:
    ValueError: raises exception if a param has len <= 0
  """
  if len(evaluated_sentences) <= 0 or len(reference_sentences) <= 0:
    raise ValueError("Collections must contain at least 1 sentence.")

  evaluated_ngrams = _get_word_ngrams(n, evaluated_sentences)
  reference_ngrams = _get_word_ngrams(n, reference_sentences)
  reference_count = len(reference_ngrams)
  evaluated_count = len(evaluated_ngrams)

  # Gets the overlapping ngrams between evaluated and reference
  overlapping_ngrams = evaluated_ngrams.intersection(reference_ngrams)
  overlapping_count = len(overlapping_ngrams)

  # Handle edge case. This isn't mathematically correct, but it's good enough
  if evaluated_count == 0:
    precision = 0.0
  else:
    precision = overlapping_count / evaluated_count

  if reference_count == 0:
    recall = 0.0
  else:
    recall = overlapping_count / reference_count

  f1_score = 2.0 * ((precision * recall) / (precision + recall + 1e-8))

  #just returning recall count in rouge, useful for our purpose
  return recall

In [19]:
# gnerate a series of tokenize predictions based off of the test dataset
tokenized_predictions = trainer.predict(test_dataset)

100%|██████████| 2/2 [00:36<00:00, 18.26s/it]


In [20]:
print(tokenized_predictions[0].shape)

(10, 20)


In [21]:
reference_sentences = []
evaluated_sentences = []

for idx, example_prediction in enumerate(tokenized_predictions[0]):
    tokens = tokenizer.decode(example_prediction, skip_special_tokens=True)
    reference_sentences.append(test_df["short_description"].iloc[idx])
    evaluated_sentences.append(tokens)

    print("reference:", reference_sentences[idx], "\nevaluated:", evaluated_sentences[idx], "\n")



reference: The former Dean of St Edmundsbury in Suffolk dies at his home after a long battle with cancer. 
evaluated: The former Dean of St Edmundsbury in Suffolk has died after a long illness. 

reference: A cyclist dies after being hit by a car outside a sports centre in the town of Kirkliston in West Lothian. 
evaluated: A cyclist has died after being hit by a car in a town in West Loth 

reference: A man who went on the run after attacking his partner is jailed for three years and three months. 
evaluated: A man who went on the run after attacking his partner has been jailed for three years 

reference: Indonesian pop singer Nazril Irham faces police questioning after homemade explicit videos were circulated online. 
evaluated: Indonesian pop singer Nazril Irham faces questioning by police after homemade explicit videos 

reference: In a study of tropical guppies, scientists find that less attractive males have "better sperm". 
evaluated: In a study of tropical guppies, scientists 

In [22]:
# run the metrics 
bleu_score = bleu(reference_sentences, evaluated_sentences)
rouge_score = rouge_n(reference_sentences, evaluated_sentences, n=2)

print("BLEU SCORE: {:.2f}, ROUGE SCORE: {:.2f}".format(bleu_score, rouge_score))


BLEU SCORE: 0.55, ROUGE SCORE: 0.54


## Evaluation

Ultimately the final tuned model performed pretty well. It received a .55 bleu score, and .54 rouge score. Above .5 by either metric indicates a pretty good degree of similarity. As for the impact of the hyperparamers on the model, I must confress that I did not play around with these enough to get a good grasp on their effect. Seeing as how it took roughly one hour to train the model each time, I only changed the parameters about twice before coming to this final model. That being said there are a few things that I think I can say based on those changes. I think that increasing the weight decay, as well as decreasing eval_steps and increasing the batch sizes definity managed to make the model train a bit faster. As for the effect that the base model had, I couldn't say for sure. My best guess as to what the effect of using a different model, say a smaller one, on the overall preformce would be a decrease in the BLEU and ROUGE scores. This would probably be because of the model not being general enough to truly summarize the text well versus a large model that had been trained on more training data.   

# Part 2: Reinforcement Learning

## Task 2

One real world application of reinforcement learning that can be modeled as an MDP is stock trading. For the sake of explanation, and my own limited understanding of stock trading, our trader will only be trading a single stock, with a limited of number of variables be taken into account. In this model, a state will only consist of two variables; the current price of the stock, and the trader's current position (The amount of said stock the trader possess). At any given momement, a trader can either buy or sell the stock. The choice of whether to buy or sell, and the related quantity make would then make up the action space of the MDP. The transision model would be a function which, the provided the trader's current state as described by their position and the current stock value, how much stock they sold or bought, and some other state returns the probability of transitioning to that newer state, provided their action and previous state. Finally, the reward in this model would be measured by the current value of the trader's position combined with whatever profit has been made from selling stock.

## Task 3

One problem in tading and finance that reinforcment learning has been applied to is risk managment when it comes to stock portfolios. More specifically, the risk-return balancing problem. This problem, as its name might imply, focuses on maximizing projected return from stocks while minimizing the risk that keeping those stocks poses. Reinforcment learning, and more specifically deep reinforcment learning can be used to solve this prblem. DeepTrader, is a model which does just this. To quote the paper in which the model was described; "DeepTrader embeds macro market conditions as an idicator to dynamically adjust the proportion between long and short funds, to lower the risk of market fluctions". Essentially the model uses broad trends in the stock market as a basis for the reward function of the reinforcment learning model.

Although I couldn't find any code for DeepTrader alone, the model is implemented in TradeMaster, an open source platform for trading which specializes in more than just portfolio managment. The TradeMaster platform is composed of 6 different modules. The first of these is a modules is comprised of the data consisting of multi-modality market data of different financial assets at multiple granularity. The second module is the data processing pipeline. The third module is comprised of multiple high-fidelity data-driven market simulators, used for mainstream quantitative trading. The fourth module is a series of implementation for RL-based trading algorithms, 13 in total. Finally there are the evaluation toolkits which incorporate 17 different measures, all of which feed into the final module; the interface designed to be used by interdisciplinary users.


DeepTrader: https://ojs.aaai.org/index.php/AAAI/article/view/16144 \\
TradeMaster: https://github.com/TradeMaster-NTU/TradeMaster?tab=readme-ov-file#trademaster-an-rl-platform-for-trading

